In [1]:
import numpy as np
import pandas as pd
import os
from pandas_plink import read_plink
import pandas_plink as pdp
from pandas_plink import get_data_folder

In [2]:
# select variants randomly
# select variants by rarity
# use linear calculation
# use non linear calculation
# with/without PC
# how rare should the phenotype be - like an existing one
# create syntetic covariates

In [52]:
GENETICS_PATH = "/sci/nosnap/michall/roeizucker/covar_tests/small_genetics"
COVARS_PATH = "/sci/archive/michall/roeizucker/covar_tests_non_coding_HADASSA_PLINK"
PLINK_PATH = "/sci/nosnap/michall/roeizucker/plink2"
TEST_COVAR = "covariates_sex_year_of_birth_AC_batch_40_PC.txt"
TEST_CHROM = "chr22_filtered_mafe-2_hwee-10_geno_02_random_001pct"
PLINK_RESULT_PATH = "/sci/archive/michall/roeizucker/covar_tests_non_coding_HADASSA_PLINK/results/chr22_very_reduced_simulation"
PLINK_RESULT_DIR = "/sci/archive/michall/roeizucker/covar_tests_non_coding_HADASSA_PLINK/results/"
PHENOTYPE_FILE_RESULT_PATH = "/sci/archive/michall/roeizucker/covar_tests_non_coding_HADASSA_PLINK/phenotypes/generated_phen.txt"
INCLUDED_COVARS = ["sex","year_of_birth","AC","batch"]
AC_VALS = ['AC_stockport_pilot','AC_manchester','AC_oxford','AC_cardiff','AC_glasgow','AC_edinburgh','AC_stoke','AC_reading','AC_bury','AC_newcastle','AC_bristol','AC_barts','AC_nottingham','AC_sheffield','AC_liverpool','AC_middlesborough','AC_hounslow','AC_croydon','AC_birmingham','AC_swansea','AC_wrexham','AC_cheadle_revisit','AC_cheadle_imaging','AC_reading_imaging','AC_newcastle_imaging']
BATCH_VALS = ['batch_Batch_b001','batch_Batch_b002','batch_Batch_b003','batch_Batch_b004','batch_Batch_b005','batch_Batch_b006','batch_Batch_b007','batch_Batch_b008','batch_Batch_b009','batch_Batch_b010','batch_Batch_b011','batch_Batch_b012','batch_Batch_b013','batch_Batch_b014','batch_Batch_b015','batch_Batch_b016','batch_Batch_b017','batch_Batch_b018','batch_Batch_b019','batch_Batch_b020','batch_Batch_b021','batch_Batch_b022','batch_Batch_b023','batch_Batch_b024','batch_Batch_b025','batch_Batch_b026','batch_Batch_b027','batch_Batch_b028','batch_Batch_b029','batch_Batch_b030','batch_Batch_b031','batch_Batch_b032','batch_Batch_b033','batch_Batch_b034','batch_Batch_b035','batch_Batch_b036','batch_Batch_b037','batch_Batch_b038','batch_Batch_b039','batch_Batch_b040','batch_Batch_b041','batch_Batch_b042','batch_Batch_b043','batch_Batch_b044','batch_Batch_b045','batch_Batch_b046','batch_Batch_b047','batch_Batch_b048','batch_Batch_b049','batch_Batch_b050','batch_Batch_b051','batch_Batch_b052','batch_Batch_b053','batch_Batch_b054','batch_Batch_b055','batch_Batch_b056','batch_Batch_b057','batch_Batch_b058','batch_Batch_b059','batch_Batch_b060','batch_Batch_b061','batch_Batch_b062','batch_Batch_b063','batch_Batch_b064','batch_Batch_b065','batch_Batch_b066','batch_Batch_b067','batch_Batch_b068','batch_Batch_b069','batch_Batch_b070','batch_Batch_b071','batch_Batch_b072','batch_Batch_b073','batch_Batch_b074','batch_Batch_b075','batch_Batch_b076','batch_Batch_b077','batch_Batch_b078','batch_Batch_b079','batch_Batch_b080','batch_Batch_b081','batch_Batch_b082','batch_Batch_b083','batch_Batch_b084','batch_Batch_b085','batch_Batch_b086','batch_Batch_b087','batch_Batch_b088','batch_Batch_b089','batch_Batch_b090','batch_Batch_b091','batch_Batch_b092','batch_Batch_b093','batch_Batch_b094','batch_Batch_b095','batch_UKBiLEVEAX_b1','batch_UKBiLEVEAX_b10','batch_UKBiLEVEAX_b11','batch_UKBiLEVEAX_b3','batch_UKBiLEVEAX_b4','batch_UKBiLEVEAX_b5','batch_UKBiLEVEAX_b6','batch_UKBiLEVEAX_b7','batch_UKBiLEVEAX_b8','batch_UKBiLEVEAX_b9']
ALL_COVARS = INCLUDED_COVARS[:-2] + AC_VALS + BATCH_VALS
NUM_PC = 40
for i in range(NUM_PC):
    ALL_COVARS.append(f"PC{i+1}")
YOB_SUBTRACTOR = 1880
YOB_DEVIDOR = 100


# phenotype generation constants
TARGET_VARIANCE = 1
DISEASE_PREVELANCE = 0.05
PHENOTYPE_DIR_RESULT_PATH = "/sci/archive/michall/roeizucker/covar_tests_non_coding_HADASSA_PLINK/phenotypes/"
INCLUDE_COVAR = True
AC_EFFECT_SCORE = 1
BATCH_EFFECT_SCORE = 1

SEX_EFFECT = 3
YOB_EFFECT = 1
PC_EFFECT_MAX = 1
# COVAR_GENETIC_EFFECT_RATIO = 2
HERETEBILITY = 0.5
COVAR_EFFECT = 0.2
# TODO change the name and functio nto heretibility to heretibility
# heretibility is the variance of the genetics / (var genetcis + var covar)

AC_EFFECT = [AC_EFFECT_SCORE]*len(AC_VALS)
BATCH_EFFECT = [BATCH_EFFECT_SCORE]*len(BATCH_VALS)
# add PC stratagy
PC_EFFECT = list(np.arange(PC_EFFECT_MAX,0,-PC_EFFECT_MAX/NUM_PC))
COVAR_EFFECTS = [SEX_EFFECT,YOB_EFFECT] + AC_EFFECT + BATCH_EFFECT + PC_EFFECT
COVAR_EFFECTS_DEVIDOR = 50 # used to reduce the effect of covaraites, can be avoided if provided covariate effects are smaller

# TODO: change to work as beta.
# create a function that accepts a vector of Betas and a flag if its binary/continues and add an option for covariates
# If its binary, add prevelance considiration
# for a person J the score for J is the sum over all betas of SNPS * the number of alleles. 
# add a noise so that the variance of phenotype will be 1 (adding noise using 1-sigma) call this Yj
# to do so calculate overall variance after simulation, and add a noise with 1- the variance (make sure that the genetic + covar variance is below 1)
# in continues state, use Yj as phenotype (maybe mul by a constant)
# in binary state the same, need to do a logit (sigmuid and then threashold or just threshold) transformation. detarmine the threshold so that X% of the population are sick
# another way, in the binary state instead of noise can do a bernuli on the result of the sigmuid (a value between 0 and 1)
# the covariats can be treated the same way the betas are used
# read about genetic architucture, compare covariates and betas effect.
# the aggregative effect of snps should be larger, but the effect of a single snp should be smaller.

# look at GWAS catalog, use a table from there for a specific disease and take their betas (find out how to calculate beta from OR)

# variant parameter
NUMBER_OF_MAJOR_VARIANTS = 20
NUMBER_OF_MEDIUM_VARIANTS = 30
NUMBER_OF_LESSER_VARIANTS = 50

MAJOR_VARIANT_EFFECT = 0.05
MEDIUM_VARIANT_EFFECT = 0.015
LESSER_VARIANT_EFFECT = 0.005

# change to betas
MAJOR_VARIANT_DELTA = 1
MEDIUM_VARIANT_DELTA = 0.2
LESSER_VARIANT_DELTA = 0.01



In [4]:
# bim, fam, G = read_plink(os.path.join(GENETICS_PATH,TEST_CHROM),verbose=True)
# # try reading only top rows?
# print("bim shape:", bim.shape)
# print("fam shape:", fam.shape)
# print("G shape:", G.shape)
# # def select_first_10_variants(bim_df):
# #     return bim_df.iloc[:10]  # first 10 rows

# # bim, fam, G = read_plink(
# #     os.path.join(GENETICS_PATH,TEST_CHROM),
# #     select_bim=select_first_10_variants
# # )

# # print(bim.shape)  # Should show 10 variants
# # print(G.shape)  

In [5]:
# G_copy = G.copy()
# bim_copy = bim.copy()
# fam_copy = fam.copy()

In [6]:
# np.random.seed(123)  # for reproducibility

# # Add a simulated 'age' covariate
# fam['age'] = np.random.normal(loc=50, scale=10, size=len(fam))

# # Add a simulated 'PC1' principal component
# fam['PC1'] = np.random.normal(loc=0, scale=1, size=len(fam))

# # If you want to define your own sex variable
# fam['sex_code'] = np.random.choice([0, 1], size=len(fam))  # 0 = Male, 1 = Female


In [7]:
# # # for val in G[0, :]:
# # #     print((val.A))
# # #     break
# # b = G[0, :]
# # c = b[0]

# def some_func(x,y):
#     return x/y
    
# try:
#     x = input()
#     y = input()
#     print(some_func(x,y))
# except Exception:
#     print("cannot divide by zero")



In [8]:
# for var in bim["snp"]:
#     print(var)
#     val_loc = list(G[0]["snp"].values).index(var)
#     specific = G.T[val_loc]
#     break

In [9]:
# # Let's pick the first variant (index 0) as our causal SNP


In [10]:
G = pdp.read_plink1_bin(os.path.join(GENETICS_PATH,TEST_CHROM) + ".bed",os.path.join(GENETICS_PATH,TEST_CHROM) + ".bim",os.path.join(GENETICS_PATH,TEST_CHROM) + ".fam")

Mapping files: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:03<00:00,  1.20s/it]


In [11]:
df = G.to_dataframe()

In [12]:
df

fid      iid father mother gender trait chrom  \
sample  variant                                                         
1688461 variant0    1688461  1688461      0      0      1    -9    22   
        variant1    1688461  1688461      0      0      1    -9    22   
        variant2    1688461  1688461      0      0      1    -9    22   
        variant3    1688461  1688461      0      0      1    -9    22   
        variant4    1688461  1688461      0      0      1    -9    22   
...                     ...      ...    ...    ...    ...   ...   ...   
1409864 variant720  1409864  1409864      0      0      1    -9    22   
        variant721  1409864  1409864      0      0      1    -9    22   
        variant722  1409864  1409864      0      0      1    -9    22   
        variant723  1409864  1409864      0      0      1    -9    22   
        variant724  1409864  1409864      0      0      1    -9    22   

                                 snp   cm       pos a0  a1  genotype  
sample  variant                                                       
1688461 variant0          rs73159809  0.0  17054795  A   G       2.0  
        variant1           rs3859824  0.0  17096707  G   C       0.0  
        variant2          rs56140010  0.0  17101672  G   A       2.0  
        variant3           rs9605005  0.0  17130268  A   G       2.0  
        variant4         rs143380912  0.0  17145480  T   A       2.0  
...                              ...  ...       ... ..  ..       ...  
1409864 variant720          rs131747  0.0  51025074  T   C       0.0  
        variant721        rs74937467  0.0  51040413  A   T       2.0  
        variant722       rs139905430  0.0  51059434  T   C       2.0  
        variant723  22:51059833_CT_C  0.0  51059833  C  CT       2.0  
        variant724         rs5770964  0.0  51073893  G   C       2.0  

[353371525 rows x 13 columns]

In [13]:
df = df.reset_index()

In [14]:
covar_df = pd.read_csv(os.path.join(COVARS_PATH,TEST_COVAR),sep="\t",header=None)


In [15]:
covar_df = covar_df.set_axis(["iid","fid"] + ALL_COVARS, axis=1)

In [16]:
legal_iid = set(covar_df["iid"].astype(str))

In [17]:
df = df[df["iid"].isin(legal_iid)]

In [18]:
df = df.reset_index()

In [20]:
# covar_df = covar_df.set_index("iid")


In [21]:
unique_variants = df["variant"].unique()

In [22]:
# TODO: change so variants are selected from all over the genom, and by prevelance 
major_variants = set(unique_variants[0:NUMBER_OF_MAJOR_VARIANTS])
medium_variants = set(unique_variants[NUMBER_OF_MAJOR_VARIANTS:NUMBER_OF_MAJOR_VARIANTS+NUMBER_OF_MEDIUM_VARIANTS])
lesser_variants = set(unique_variants[NUMBER_OF_MAJOR_VARIANTS+NUMBER_OF_MEDIUM_VARIANTS:NUMBER_OF_MAJOR_VARIANTS+NUMBER_OF_MEDIUM_VARIANTS+NUMBER_OF_LESSER_VARIANTS])

In [23]:
included_variants = major_variants.union(medium_variants).union(lesser_variants)
# TODO: add protective variants

In [24]:
df = df[df["variant"].isin(included_variants)]

In [25]:
genotype_dict = df[["iid","variant","genotype"]].set_index(["iid","variant"]).to_dict(orient="index")
for key in genotype_dict.keys():
    genotype_dict[key] = genotype_dict[key]["genotype"]

In [26]:
effect_dict = {}
for var in included_variants:
    if var in major_variants:
        effect_dict[var] = MAJOR_VARIANT_DELTA
        continue
    if var in medium_variants:
        effect_dict[var] = MEDIUM_VARIANT_DELTA
        continue
    effect_dict[var] = LESSER_VARIANT_DELTA

In [27]:
# %%time
df["effect"] = df["variant"].map(effect_dict)
# df["effect"] = df["effect"]**df["genotype"]

# Delta form
df["effect"] = df["effect"]*df["genotype"]

In [28]:
df = df.reset_index()

In [29]:
# # TODO: change so that the variance is for the phenotype_scroe
# added_variance = TARGET_VARIANCE -df["effect"].var()
# mu, sigma = 0, added_variance # mean and standard deviation
# noise = np.random.normal(mu, sigma**0.5, len(df))
# df["fixed_effect"] = df["effect"] + noise
# df["fixed_effect"].var()

In [97]:

covar_df["year_of_birth"] = 1- ((np.array(covar_df[covar_df.columns[2:]])[:,1] - YOB_SUBTRACTOR)/YOB_DEVIDOR)
covar_df["year_of_birth"].describe()

count    3.333710e+05
mean     1.960396e+01
std      7.983349e-12
min      1.960396e+01
25%      1.960396e+01
50%      1.960396e+01
75%      1.960396e+01
max      1.960396e+01
Name: year_of_birth, dtype: float64

In [31]:
covar_df["covar_effects_chance"] = ((np.array(covar_df[covar_df.columns[2:]]) @ (np.array(COVAR_EFFECTS)/ COVAR_EFFECTS_DEVIDOR))) 

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 172 is different from 176)

In [32]:
# TODO: change name
disease_chance_dict = (1 - df.groupby("iid")["effect"].sum()).to_dict()
# disease_chance_dict
# TODO: change name, this is not a chance, but a result
covar_df["genetic_chance"] = covar_df["iid"].astype(str).map(disease_chance_dict)


In [50]:
# HERETEBILITY = var(ax) = a**2*var(x)
# HERETEBILITY = a**2*var(x)
# a**2 = HERETEBILITY/var(x)
# a = sqrt(HERETEBILITY/var(x))

In [85]:
# adjust so genetic effect and covar effect match the requested
gene_var = ((covar_df["genetic_chance"]).var())
covar_var = (covar_df["covar_effects_chance"].var())
genetic_factor = np.sqrt(HERETEBILITY/gene_var)
covar_factor = np.sqrt(COVAR_EFFECT/covar_var)
# x = COVAR_GENETIC_EFFECT_RATIO*covar_sum/gene_sum
# covar_df["genetic_chance"] = covar_df["genetic_chance"] * x
covar_df["genetic_chance"] = covar_df["genetic_chance"] * genetic_factor
covar_df["covar_effects_chance"] = covar_df["covar_effects_chance"] * covar_factor

In [89]:
covar_df["disease_chance"] = covar_df["genetic_chance"] + covar_df["covar_effects_chance"]

In [90]:
# TODO: change so that the variance is for the phenotype_scroe
added_variance = TARGET_VARIANCE -covar_df["disease_chance"].var()

mu, sigma = 0, added_variance # mean and standard deviation
noise = np.random.normal(mu, sigma**0.5, len(covar_df))
covar_df["disease_chance"] = covar_df["disease_chance"] + noise
covar_df["disease_chance"].var()


0.9988117373679034

count    332645.000000
mean        -13.316132
std           0.999406
min         -17.410824
25%         -14.002057
50%         -13.349280
75%         -12.662661
max          -7.852709
Name: disease_chance, dtype: float64

In [36]:
quantile = (covar_df[["disease_chance"]].quantile(1 - DISEASE_PREVELANCE, method="table", interpolation="nearest")["disease_chance"])
covar_df['disease_indicator'] = (
    covar_df['disease_chance'] > quantile
).astype(int).astype(float)

In [37]:
# TODO:
# randomly select variants instead of first 100
# multiple phenotypes with different heretability/covar/random effects
#     for each phenotype, run on each covar file that appeard in the original study
# next step - use snp from catalog
# when evaluating, finding something close to causal, is not a mistake, should be treated as true positive


In [38]:
# AC_EFFECT_SCORE = 1
# BATCH_EFFECT_SCORE = 1
# SEX_EFFECT = 3
# YOB_EFFECT = 1
# PC_EFFECT_MAX = 1


In [39]:
phenotype_file_result_name = (
    f"generated_pehn_result_target_var={TARGET_VARIANCE}_target_prev={DISEASE_PREVELANCE}" + 
    f"major_n={NUMBER_OF_MAJOR_VARIANTS}_med_n={NUMBER_OF_MEDIUM_VARIANTS}_less__m={NUMBER_OF_LESSER_VARIANTS}" +
    f"major_d={MAJOR_VARIANT_DELTA}_med_d={MEDIUM_VARIANT_DELTA}_less_d={LESSER_VARIANT_DELTA}" 
    f"gene_covar_effect_ratio={COVAR_GENETIC_EFFECT_RATIO}"
    )
if INCLUDE_COVAR:
    phenotype_file_result_name+= (
        f"_ac={AC_EFFECT_SCORE}_batch={BATCH_EFFECT_SCORE}_sex={SEX_EFFECT}_yob={YOB_EFFECT}_pc={PC_EFFECT_MAX}"
    )

phenotype_file_result_name = phenotype_file_result_name+".txt"
phenotype_file_path = os.path.join(PHENOTYPE_DIR_RESULT_PATH,phenotype_file_result_name)
covar_df[["iid","fid","disease_indicator"]].to_csv(phenotype_file_path,index=False,header=False,sep="\t")

In [40]:
phenotype_file_result_name

'generated_pehn_result_target_var=1_target_prev=0.05major_n=20_med_n=30_less__m=50major_d=1_med_d=0.2_less_d=0.01gene_covar_effect_ratio=2_ac=1_batch=1_sex=3_yob=1_pc=1.txt'

In [41]:
chrom_path = os.path.join(GENETICS_PATH,TEST_CHROM)
result_path = os.path.join(PLINK_RESULT_DIR,phenotype_file_result_name.replace(".txt",""))

In [42]:
command =f"{PLINK_PATH} --bed {chrom_path}.bed --bim {chrom_path}.bim --fam {chrom_path}.fam --pheno {phenotype_file_path} --covar {os.path.join(COVARS_PATH,TEST_COVAR)}  --out  {result_path} --glm hide-covar --mac 20 --covar-variance-standardize --freq --threads 30 --memory 30000 --1"

In [43]:
print(command)

/sci/nosnap/michall/roeizucker/plink2 --bed /sci/nosnap/michall/roeizucker/covar_tests/small_genetics/chr22_filtered_mafe-2_hwee-10_geno_02_random_001pct.bed --bim /sci/nosnap/michall/roeizucker/covar_tests/small_genetics/chr22_filtered_mafe-2_hwee-10_geno_02_random_001pct.bim --fam /sci/nosnap/michall/roeizucker/covar_tests/small_genetics/chr22_filtered_mafe-2_hwee-10_geno_02_random_001pct.fam --pheno /sci/archive/michall/roeizucker/covar_tests_non_coding_HADASSA_PLINK/phenotypes/generated_pehn_result_target_var=1_target_prev=0.05major_n=20_med_n=30_less__m=50major_d=1_med_d=0.2_less_d=0.01gene_covar_effect_ratio=2_ac=1_batch=1_sex=3_yob=1_pc=1.txt --covar /sci/archive/michall/roeizucker/covar_tests_non_coding_HADASSA_PLINK/covariates_sex_year_of_birth_AC_batch_40_PC.txt  --out  /sci/archive/michall/roeizucker/covar_tests_non_coding_HADASSA_PLINK/results/generated_pehn_result_target_var=1_target_prev=0.05major_n=20_med_n=30_less__m=50major_d=1_med_d=0.2_less_d=0.01gene_covar_effect_ra

In [44]:
participants = set(covar_df["fid"].astype(str))

In [45]:
reset_df[reset_df["sample"].isin(participants)]

NameError: name 'reset_df' is not defined

In [ ]:
np.array(covar_df[covar_df.columns[2:-2]])[0]